In [8]:
#Chargement des données
# Data
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os 
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt


# Graphics
import seaborn as sns ; sns.set()

# Machine learning - Preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer

# Machine learning - Automatisation
from sklearn.pipeline import Pipeline
from sklearn import set_config

# Machine learning - Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Machine learning - Model selection
from sklearn.model_selection import train_test_split, GridSearchCV



import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler

In [9]:
load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

SQL = '''SELECT "averageRating", "startYear", "runtimeMinutes" 
        from principal.title_basics join principal.title_ratings 
        on principal.title_basics."tconst" = principal.title_ratings."tconst"
        where principal.title_basics."runtimeMinutes" Is NOT NULL  
        limit 50000;'''
data = pd.read_sql(SQL, engine)

In [10]:
data["averageRating"].fillna(data["averageRating"].mean(), inplace=True)
data["startYear"].fillna(data["startYear"].mean(), inplace=True)
data["runtimeMinutes"].fillna(data["runtimeMinutes"].mean(), inplace=True)
data

,averageRating,startYear,runtimeMinutes
0,5.3,2001,96
1,7.0,1956,91
2,4.4,1953,26
3,6.7,1970,85
4,5.3,1967,90
...,...,...,...
49995,5.3,2004,87
49996,6.8,2005,91
49997,7.0,2004,270
49998,5.7,2005,92


In [11]:
# Séparation des caractéristiques et de la cible
X = data.drop(columns=["averageRating"])
y = data["averageRating"]

# Séparation des données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [12]:
# Création des pipelines pour chaque modèle
pipelines = {
    'Linear Regression': Pipeline([('scaler', StandardScaler()), ('regressor', LinearRegression())]),
    'Ridge Regression': Pipeline([('scaler', StandardScaler()), ('regressor', Ridge())]),
    'Lasso Regression': Pipeline([('scaler', StandardScaler()), ('regressor', Lasso())]),
    'ElasticNet': Pipeline([('scaler', StandardScaler()), ('regressor', ElasticNet())]),
    'Random Forest Regression': Pipeline([('regressor', RandomForestRegressor())]),
    'Gradient Boosting Regression': Pipeline([('regressor', GradientBoostingRegressor())]),
    'SVR': Pipeline([('scaler', StandardScaler()), ('regressor', SVR())]),
}

# Paramètres pour GridSearchCV pour chaque modèle
parameters = {
    'Linear Regression': {},
    'Ridge Regression': {'regressor__alpha': [0.1, 1.0, 10.0]},
    'Lasso Regression': {'regressor__alpha': [0.1, 1.0, 10.0]},
    'ElasticNet': {'regressor__alpha': [0.1, 1.0, 10.0], 'regressor__l1_ratio': [0.1, 0.5, 0.9]},
    'Random Forest Regression': {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]},
    'Gradient Boosting Regression': {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [3, 5, 7]},
    'SVR': {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1.0, 10.0]},
}


In [13]:

# Boucle sur les modèles pour ajuster avec GridSearchCV
best_models = {}
for model_name, pipeline in pipelines.items():
    print(f"----------------{model_name}--------------------")
    
    grid_search = GridSearchCV(pipeline, parameters[model_name], scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)
    
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best {model_name} score: {-grid_search.best_score_}")



Training Linear Regression...
Best parameters: {}
Best Linear Regression score: 1.872397587418316
Training Ridge Regression...
Best parameters: {'regressor__alpha': 10.0}
Best Ridge Regression score: 1.8723974719357226
Training Lasso Regression...
Best parameters: {'regressor__alpha': 0.1}
Best Lasso Regression score: 1.8790797494375
Training Random Forest Regression...
Best parameters: {'regressor__max_depth': 10, 'regressor__n_estimators': 200}
Best Random Forest Regression score: 1.7043177208886835


In [14]:
# Evaluation des modèles sur les données de test
print("\nEvaluation on test data:")
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{model_name} MSE: {mse}")


Evaluation on test data:
Linear Regression MSE: 1.953602455011641
Ridge Regression MSE: 1.9536031886483833
Lasso Regression MSE: 1.9631146474999999
Random Forest Regression MSE: 1.7531854918411232
